In [ ]:
# faceMask hDDM face ratings analysis script: mask analyses of false negatives (failing to ID correct emotion)
# 12/07/20

# note: to be run within python 3.5 environment named "hddm" (from console: 'source activate hddm')
# confirm jupyter notebook is launching from "hddm" environment (see top-right: 'conda env:hddm')

# set up
import numpy as np
print(np.__version__) # should be 1.11.3

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from patsy import dmatrix

import hddm
print(hddm.__version__) # should be 0.6.0

In [ ]:
# load hDDM-processed data from CSV file into a NumPy structured array
data = hddm.load_csv('faceMask2_hddm_proc.csv')
                     
# flip 'no' response RTs to be negative
data = hddm.utils.flip_errors(data)

# check dataset
data.head(10)

In [ ]:
# plot RTs for each participant
fig = plt.figure()
ax = fig.add_subplot(111, xlabel='RT', ylabel='count', title='RT distributions')
for i, subj_data in data.groupby('subj_idx'):
    subj_data.rt.hist(bins=20, histtype='step', ax=ax)

In [ ]:
# subset dataset by emotion rating type (fearful) and facial expression (angry)
fa_data = data.loc[(data['emotionRating'] == 'fearful') & (data['expression'] == 'angry')]
fa_data.head(10)

In [ ]:
# drop any subjects missing columns in design matrix (otherwise produces error)
fa_data = fa_data[(fa_data.subj_idx != 'dqph4ch0mkofejw') & (fa_data.subj_idx != 'hdn2l3df216jykn') & (fa_data.subj_idx != 'tz7c8yczxjyfo0y') & (fa_data.subj_idx != 'xs5439nm2v85thb')]

In [ ]:
# define model
m_fa = hddm.HDDMRegressor(fa_data, "v ~ C(mask)", bias=True, p_outlier=0.05)

In [ ]:
# run model
m_fa.sample(5000, burn=200, dbname='traces.db', db='pickle')
m_fa.print_stats()
m_fa.plot_posteriors()

In [ ]:
# save model stats
fa_stats = m_fa.gen_stats()
print(fa_stats)
fa_stats.to_csv('fa_5000/faceMask_hddm_drift_maskFalsePositives_fa_5000.csv', index=True)

# save model posterior plots
m_fa.plot_posteriors(path='fa_5000/_posteriors', save=True)

In [ ]:
# plot model posteriors by mask
fa_v_none, fa_v_lower, fa_v_upper = m_fa.nodes_db.node[["v_Intercept", "v_C(mask)[T.lower]", "v_C(mask)[T.upper]"]]
hddm.analyze.plot_posterior_nodes([fa_v_none, fa_v_lower, fa_v_upper])
plt.savefig('fa_5000/faceMask_hddm_drift_maskFalsePositives_fa_5000_v_mask.pdf')

In [ ]:
## estimate probabilities that mask coefficient posteriors differ from 0
# note that comparison condition coefficients (i.e. lower/upper) are *relative to* baseline condition (i.e. no mask)
# for lower/upper, =0 means no change from baseline, <0 means less than baseline, and >0 means greater than baseline
# for no mask, =0 means null drift, <0 means negative drift, and >0 means positive drift
print("P(fa_v_none < 0) = ", (fa_v_none.trace() < 0).mean())
print("P(fa_v_lower < 0) = ", (fa_v_lower.trace() < 0).mean())
print("P(fa_v_upper > 0) = ", (fa_v_upper.trace() > 0).mean())

# estimate probability that lower and upper mask coefficient posteriors differ
print("P(fa_v_lower < fa_v_upper) = ", (fa_v_lower.trace() < fa_v_upper.trace()).mean())

In [ ]:
# subset dataset by emotion rating type (fearful) and facial expression (sad)
fs_data = data.loc[(data['emotionRating'] == 'fearful') & (data['expression'] == 'sad')]
fs_data.head(10)

In [ ]:
# drop any subjects missing columns in design matrix (otherwise produces error)
fs_data = fs_data[(fs_data.subj_idx != 'dqph4ch0mkofejw') & (fs_data.subj_idx != 'xs5439nm2v85thb')]

In [ ]:
# define model
m_fs = hddm.HDDMRegressor(fs_data, "v ~ C(mask)", bias=True, p_outlier=0.05)

In [ ]:
# run model
m_fs.sample(5000, burn=200, dbname='traces.db', db='pickle')
m_fs.print_stats()
m_fs.plot_posteriors()

In [ ]:
# save model stats
fs_stats = m_fs.gen_stats()
print(fs_stats)
fs_stats.to_csv('fs_5000/faceMask_hddm_drift_maskFalsePositives_fs_5000.csv', index=True)

# save model posterior plots
m_fs.plot_posteriors(path='fs_5000/_posteriors', save=True)

In [ ]:
# plot model posteriors by mask
fs_v_none, fs_v_lower, fs_v_upper = m_fs.nodes_db.node[["v_Intercept", "v_C(mask)[T.lower]", "v_C(mask)[T.upper]"]]
hddm.analyze.plot_posterior_nodes([fs_v_none, fs_v_lower, fs_v_upper])
plt.savefig('fs_5000/faceMask_hddm_drift_maskFalsePositives_fs_5000_v_mask.pdf')

In [ ]:
## estimate probabilities that mask coefficient posteriors differ from 0
# note that comparison condition coefficients (i.e. lower/upper) are *relative to* baseline condition (i.e. no mask)
# for lower/upper, =0 means no change from baseline, <0 means less than baseline, and >0 means greater than baseline
# for no mask, =0 means null drift, <0 means negative drift, and >0 means positive drift
print("P(fs_v_none < 0) = ", (fs_v_none.trace() < 0).mean())
print("P(fs_v_lower < 0) = ", (fs_v_lower.trace() < 0).mean())
print("P(fs_v_upper > 0) = ", (fs_v_upper.trace() > 0).mean())

# estimate probability that lower and upper mask coefficient posteriors differ
print("P(fs_v_lower < fs_v_upper) = ", (fs_v_lower.trace() < fs_v_upper.trace()).mean())

In [ ]:
# subset dataset by emotion rating type (fearful) and facial expression (surprised)
fr_data = data.loc[(data['emotionRating'] == 'fearful') & (data['expression'] == 'surprised')]
fr_data.head(10)

In [ ]:
# drop any subjects missing columns in design matrix (otherwise produces error)
fr_data = fr_data[(fr_data.subj_idx != 'dqph4ch0mkofejw') & (fr_data.subj_idx != 'xs5439nm2v85thb')]

In [ ]:
# define model
m_fr = hddm.HDDMRegressor(fr_data, "v ~ C(mask)", bias=True, p_outlier=0.05)

In [ ]:
# run model
m_fr.sample(5000, burn=200, dbname='traces.db', db='pickle')
m_fr.print_stats()
m_fr.plot_posteriors()

In [ ]:
# save model stats
fr_stats = m_fr.gen_stats()
print(fr_stats)
fr_stats.to_csv('fr_5000/faceMask_hddm_drift_maskFalsePositives_fr_5000.csv', index=True)

# save model posterior plots
m_fr.plot_posteriors(path='fr_5000/_posteriors', save=True)

In [ ]:
# plot model posteriors by mask
fr_v_none, fr_v_lower, fr_v_upper = m_fr.nodes_db.node[["v_Intercept", "v_C(mask)[T.lower]", "v_C(mask)[T.upper]"]]
hddm.analyze.plot_posterior_nodes([fr_v_none, fr_v_lower, fr_v_upper])
plt.savefig('fr_5000/faceMask_hddm_drift_maskFalsePositives_fr_5000_v_mask.pdf')

In [ ]:
## estimate probabilities that mask coefficient posteriors differ from 0
# note that comparison condition coefficients (i.e. lower/upper) are *relative to* baseline condition (i.e. no mask)
# for lower/upper, =0 means no change from baseline, <0 means less than baseline, and >0 means greater than baseline
# for no mask, =0 means null drift, <0 means negative drift, and >0 means positive drift
print("P(fr_v_none < 0) = ", (fr_v_none.trace() < 0).mean())
print("P(fr_v_lower < 0) = ", (fr_v_lower.trace() < 0).mean())
print("P(fr_v_upper > 0) = ", (fr_v_upper.trace() > 0).mean())

# estimate probability that lower and upper mask coefficient posteriors differ
print("P(fr_v_lower < fr_v_upper) = ", (fr_v_lower.trace() < fr_v_upper.trace()).mean())